** Introducción al ejercicio y experiencia de tomar el curso (motivaciones, desafios y conclusiones) **

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [ ]:
nhl_df


### wikipedia_data.html

El archivo wikipedia_data.html contiene una tabla con información sobre regiones metropolitanas y los equipos deportivos asociados a ellas. Este archivo es el "source of authority" para determinar la localización de los equipos deportivos, lo que significa que cualquier equipo debe ser mapeado a la región metropolitana específica mencionada en este archivo.

#### Contenido de wikipedia_data.html
    Regiones Metropolitanas: Estas son áreas urbanas grandes que generalmente abarcan varias ciudades o condados. El nombre de la región metropolitana es la clave para agrupar equipos deportivos.

    Equipos Deportivos: La tabla incluye equipos de los "Big 4" deportes profesionales en América del Norte: NFL (fútbol americano), MLB (béisbol), NBA (baloncesto) y NHL (hockey sobre hielo).

    Población: También incluye estimaciones de la población de estas regiones metropolitanas, que son necesarias para calcular la correlación con los ratios de victorias/derrotas de los equipos deportivos.

#### Estructura de la Tabla

El DataFrame que se obtiene de wikipedia_data.html normalmente tiene las siguientes columnas:

    (Metropolitan area) Nombre de la Región Metropolitana: El nombre de la región, como "San Francisco Bay Area".
    (Population (2016 est.)[8]) Población (estimación de 2016): La población estimada de la región en el año 2016.
    (NFL) Equipos de la NFL: Equipos de fútbol americano asociados con la región.
    (MLB) Equipos de la MLB: Equipos de béisbol asociados con la región.
    (NBA) Equipos de la NBA: Equipos de baloncesto asociados con la región.
    (NHL) Equipos de la NHL: Equipos de hockey sobre hielo asociados con la región.

## Pregunta 1
Para esta pregunta, calcula el ratio de correlación de victorias/derrotas con la población de la ciudad para NHL usando datos de 2018

In [ ]:
# Creamos un nuevo dataframe para la info de las ciudades, poblacion y equipos

cities["NHL"] = cities["NHL"].str.replace(r"\[.+\]", "", regex=True)
cities["NHL"] = cities["NHL"].replace({
    "Rangers Islanders Devils": "Rangers,Islanders,Devils",
    "Kings Ducks": "Kings,Ducks"
})
cities["NHL"] = cities["NHL"].apply(lambda x: x.split(','))
cities = cities.explode("NHL")

infocities = cities.iloc[:, [0,1,5]].copy()

infocities.rename(columns={0: 'Metropolitan area', 1: 'Population'}, inplace=True)
infocities.columns = ['Metropolitan area', 'Population', 'NHL']
infocities = infocities[infocities['NHL'].str.contains(r'[a-zA-Z]')]
infocities["NHL"]

In [ ]:
# Creamos un dataframe para la info de los equipos y sus victorias y derrotas
info_league = nhl_df[nhl_df['year'] == 2018][['team', 'W', 'L']]
info_league['team'] = info_league['team'].str.replace('*', '')
info_league = info_league[~info_league['team'].str.contains('Division')]
info_league

In [ ]:
teams_shorts = infocities["NHL"].str.strip()
teams_complete = info_league["team"].str.strip()

final_list = []

# Verificar si hay coincidencias parciales
for short_name in teams_shorts:
# Encontrar el índice donde se encuentra la coincidencia
    idx = teams_complete[teams_complete.str.contains(short_name)].index
    if not idx.empty:
        index = idx[0]
        #print(f"Match found for '{short_name}', corresponding full team name: {teams_complete[index]}")
        
        # Acceder a la fila correspondiente en info_league
        team_complete_row = info_league.loc[index]

        # Acceder a la fila correspondiente en infocities
        infocities_row = infocities[infocities['NHL'] == short_name].iloc[0]

               # Combinar las filas en una sola con pd.concat()
        combined_row = pd.concat([team_complete_row, infocities_row])

        # Agregar la fila combinada a la lista
        final_list.append(combined_row)

        # Imprimir la fila combinada
        #print(combined_row)
        #print("--------------------------------------------------------")

final_df = pd.DataFrame(final_list)
final_df


In [98]:
result_df = final_df.copy()

result_df['W'] = pd.to_numeric(result_df['W'])
result_df['L'] = pd.to_numeric(result_df['L'])

result_df['Win-Loss Ratio'] = result_df['W'] / (result_df['W'] + result_df['L'])
result_df = result_df.drop(["team", "NHL"], axis=1)

# print(len(result_df.drop_duplicates(subset=["Metropolitan area"]))) # Ya son 28 valores

result_df

# Eliminar filas duplicadas en la columna 'Metropolitan area'
df_unique = result_df.drop_duplicates(subset=['Metropolitan area'])

# Calcular la media del Win-Loss Ratio por cada área metropolitana
mean_ratio = result_df.groupby('Metropolitan area')['Win-Loss Ratio'].mean().reset_index()
mean_ratio

population_data = result_df.groupby('Metropolitan area')['Population'].first().reset_index()
population_data

mean_ratio_with_population = pd.merge(mean_ratio, population_data, on='Metropolitan area', how='left')
mean_ratio_with_population

mean_ratio_with_population['Population'] = mean_ratio_with_population['Population'].astype(float)

population_by_region = mean_ratio_with_population['Population']
win_loss_by_region = mean_ratio_with_population['Win-Loss Ratio'] 

final = stats.pearsonr(population_by_region, win_loss_by_region)[0]
print(final)



0.012486162921209923


In [ ]:
population_by_region = mean_ratio_with_population['Population']
win_loss_by_region = mean_ratio_with_population['Win-Loss Ratio'] 

final = stats.pearsonr(population_by_region, win_loss_by_region)[0]
print(final)

### AHHHSHAGDGAGDA


In [ ]:

result_df = final_df
result_df['W'] = pd.to_numeric(result_df['W'])
result_df['L'] = pd.to_numeric(result_df['L'])

# Calcular el ratio de victorias/derrotas
result_df['Win_loss_ratio'] = result_df['W'] / (result_df['W'] + result_df['L'])

# Calcular la correlación entre el ratio de victorias/derrotas y la población
correlation = result_df['Win_loss_ratio'].corr(result_df['Population'])

print(f"El coeficiente de correlación entre el ratio de victorias/derrotas y la población es: {correlation}")

In [96]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def get_area(team):
    #print(team)
    #print(nhl_cities.index.values)
    for each in list(nhl_cities.index.values):
        #print(each)
        if team in each:
            #print(team)
           # print(nhl_cities.at[each, 'Metropolitan area'])
            #print(nhl_cities.at[each, 'Metropolitan area'])
            return nhl_cities.at[each, 'Metropolitan area']


nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
#print(nhl_df['team'])
#print(cities[['Metropolitan area','NHL']])
#print(cities[cities['NHL'].str.contains('Devils')])
#print(cities.shape)
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

population =cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population['Metropolitan area']=population['Metropolitan area'].str.strip()
population.rename(columns={'Population (2016 est.)[8]':'Population'},inplace=True)
population = population.set_index('Metropolitan area')
cities['NHL'].replace(r'(.*)\[.*\].*|[-—]',r'\1',regex=True, inplace=True)
cities['NHL'].replace('', np.nan, inplace=True)
cities.dropna(inplace=True)

nhl_cities = cities[['Metropolitan area', 'NHL']].set_index('NHL')

#print(nhl_cities)
nhl_df= nhl_df[nhl_df['year'] == 2018].drop([0, 9, 18, 26], axis=0).drop(['League','year','RPt%','ROW','PTS%','PTS',
                                                                          'GP','OL','GF','GA','SRS','SOS'],axis=1)
nhl_df['team'].replace(r'(.*)[\*].*',r"\1".strip(),regex=True, inplace=True)
nhl_df['Area']=nhl_df['team'].str.split(" ").str[-1:]
#print(nhl_df['Area'].str[0])
#nhl_df["Area"]= nhl_df["Area"].str[0].str.cat(nhl_df['Area'].str[1], sep =" ",na_rep ="")
nhl_df["Area"]= nhl_df["Area"].str[0]
nhl_df['Area'] = nhl_df['Area'].apply(lambda x: get_area(x))

#print(nhl_df)
#print(nhl_df)
#nhl_df['W']=pd.to_numeric(nhl_df['W'])
#nhl_df['L']=pd.to_numeric(nhl_df['L'])

nhl_df[['W','L']] = nhl_df[['W','L']].apply(pd.to_numeric, axis=1)

nhl_df=nhl_df.groupby('Area').sum()
#print(nhl_df)

#nhl_df['Ratio']=nhl_df['W'] / (nhl_df['W']+ nhl_df['L'])
#nhl_df=nhl_df.assign(Ratio=lambda x: x['W'] / (x['W'] + x['L']))
nhl_df.eval("Ratio =W / (W + L)", inplace=True)

#nhl_df.set_index('Area',inplace=True)
nhl_df.drop(['W','L'], axis=1,inplace=True)
#print(nhl_df)
#print(population['Population'])
#print(nhl_df['Ratio'])
out_df = pd.merge(nhl_df, population, how="inner", left_index=True, right_index=True)
out_df['Population']=pd.to_numeric(out_df['Population'])
#print(out_df)
#print(out_df['Ratio'])

def nhl_correlation(): 
    # YOUR CODE HERE
    
    #raise NotImplementedError()
    
    population_by_region = out_df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region =out_df['Ratio']  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    #Oakland Raiders
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


/var/folders/bw/_pgpsw6n67z55_fnwxmypghr0000gn/T/ipykernel_62953/83909568.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population['Metropolitan area']=population['Metropolitan area'].str.strip()
/var/folders/bw/_pgpsw6n67z55_fnwxmypghr0000gn/T/ipykernel_62953/83909568.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population.rename(columns={'Population (2016 est.)[8]':'Population'},inplace=True)


0.012308996455744264

In [ ]:
out_df